In [ ]:
!pip install pennylane

In [ ]:
import pennylane as qml
from pennylane import numpy as np

# --- 1. Define the Quantum Device ---
# We need 3 qubits for the teleportation protocol
dev = qml.device("default.qubit", wires=3)

# --- 2. Define the Core Teleportation Logic ---
# This function contains all the steps of the teleportation
# *except* for the final measurement, which we will change.

def teleport_operations(alpha, beta):
    """
    Applies all gates for the teleportation protocol.
    This is a helper function, not a QNode.
    """
    # === STEP 1: PREPARE MESSAGE (Alice) ===
    # Alice creates an arbitrary quantum state on wire 0
    qml.RY(alpha, wires=0)
    qml.RZ(beta, wires=0)

    # === STEP 2: CREATE BELL PAIR (Alice & Bob) ===
    # Create an entangled Bell state on wires 1 and 2
    qml.Hadamard(wires=1)
    qml.CNOT(wires=[1, 2])

    # === STEP 3: ALICE'S BELL MEASUREMENT ===
    # Alice entangles her message (0) with her Bell pair half (1)
    qml.CNOT(wires=[0, 1])
    qml.Hadamard(wires=0)

    # === STEP 4: ALICE'S MEASUREMENT ===
    # Alice measures her qubits (0 and 1)
    m0 = qml.measure(wires=0)
    m1 = qml.measure(wires=1)

    # === STEP 5: BOB'S CORRECTION ===
    # Bob applies corrections to his qubit (2) based on
    # Alice's classical measurement results.
    qml.cond(m1, qml.X)(wires=2)
    qml.cond(m0, qml.Z)(wires=2)

# --- 3. Define the Verification Circuits ---

# We need 3 circuits to measure the "teleported" state (Bob's qubit)
# in the X, Y, and Z bases.

@qml.qnode(dev)
def teleport_and_measure_Z(alpha, beta):
    """Teleports the state and measures Bob's qubit in the Z basis."""
    teleport_operations(alpha, beta)
    # Return the Z-basis expectation value of Bob's qubit (wire 2)
    return qml.expval(qml.PauliZ(wires=2))

@qml.qnode(dev)
def teleport_and_measure_X(alpha, beta):
    """Teleports the state and measures Bob's qubit in the X basis."""
    teleport_operations(alpha, beta)
    # Return the X-basis expectation value of Bob's qubit (wire 2)
    return qml.expval(qml.PauliX(wires=2))

@qml.qnode(dev)
def teleport_and_measure_Y(alpha, beta):
    """Teleports the state and measures Bob's qubit in the Y basis."""
    teleport_operations(alpha, beta)
    # Return the Y-basis expectation value of Bob's qubit (wire 2)
    return qml.expval(qml.PauliY(wires=2))


# --- 4. Define the "Ground Truth" Message Circuits ---

# We also need 3 circuits to get the "original" message's
# expectation values. This is our "ground truth" to compare against.
dev_msg = qml.device("default.qubit", wires=1)

@qml.qnode(dev_msg)
def original_message_expval_Z(alpha, beta):
    """Creates the original message and measures in the Z basis."""
    qml.RY(alpha, wires=0)
    qml.RZ(beta, wires=0)
    return qml.expval(qml.PauliZ(wires=0))

@qml.qnode(dev_msg)
def original_message_expval_X(alpha, beta):
    """Creates the original message and measures in the X basis."""
    qml.RY(alpha, wires=0)
    qml.RZ(beta, wires=0)
    return qml.expval(qml.PauliX(wires=0))

@qml.qnode(dev_msg)
def original_message_expval_Y(alpha, beta):
    """Creates the original message and measures in the Y basis."""
    qml.RY(alpha, wires=0)
    qml.RZ(beta, wires=0)
    return qml.expval(qml.PauliY(wires=0))

# --- 5. Verification Script ---
if __name__ == "__main__":

    # Define arbitrary angles for the message state
    alpha = 0.543
    beta = 1.234

    print("--- Quantum Teleportation Simulation with PennyLane ---")
    print(f"Original message parameters: alpha={alpha}, beta={beta}\n")

    # === Get the "Expected" Expectation Values ===
    exp_Z_original = original_message_expval_Z(alpha, beta)
    exp_X_original = original_message_expval_X(alpha, beta)
    exp_Y_original = original_message_expval_Y(alpha, beta)

    print("--- Original Message Expectation Values ---")
    print(f"<Z> = {exp_Z_original:.6f}")
    print(f"<X> = {exp_X_original:.6f}")
    print(f"<Y> = {exp_Y_original:.6f}\n")

    # === Get the "Teleported" Expectation Values ===
    exp_Z_teleported = teleport_and_measure_Z(alpha, beta)
    exp_X_teleported = teleport_and_measure_X(alpha, beta)
    exp_Y_teleported = teleport_and_measure_Y(alpha, beta)

    print("--- Teleported State Expectation Values (from Bob's Qubit) ---")
    print(f"<Z> = {exp_Z_teleported:.6f}")
    print(f"<X> = {exp_X_teleported:.6f}")
    print(f"<Y> = {exp_Y_teleported:.6f}\n")

    # === Compare the Expectation Values ===
    success_Z = np.allclose(exp_Z_original, exp_Z_teleported)
    success_X = np.allclose(exp_X_original, exp_X_teleported)
    success_Y = np.allclose(exp_Y_original, exp_Y_teleported)

    print("--- Verification ---")
    print(f"Z-values match: {success_Z}")
    print(f"X-values match: {success_X}")
    print(f"Y-values match: {success_Y}\n")

    if success_Z and success_X and success_Y:
        print("✅ SUCCESS: The teleported state is identical to the original state.")
    else:
        print("❌ FAILURE: The states do not match.")

--- Quantum Teleportation Simulation with PennyLane ---
Original message parameters: alpha=0.543, beta=1.234

--- Original Message Expectation Values ---
<Z> = 0.856162
<X> = 0.170754
<Y> = 0.487677

--- Teleported State Expectation Values (from Bob's Qubit) ---
<Z> = 0.856162
<X> = 0.170754
<Y> = 0.487677

--- Verification ---
Z-values match: True
X-values match: True
Y-values match: True

✅ SUCCESS: The teleported state is identical to the original state.
